This lecture introduces GPU computing in Julia.

## GPGPU

GPUs are ubiquitous in modern computers. Following are NVIDIA GPUs on today's typical computer systems.

| NVIDIA GPUs         | H100 PCIe                           | RTX 6000                                 | RTX 5000                              |
|---------------------|----------------------------------------|-----------------------------------------|--------------------------------------|
|                     | ![H100](nvidia_h100.png) | ![RTX 6000](nvidia_rtx6000.png)    | ![RTX 5000](nvidia_rtx5000.png) |
| Computers           | servers, cluster                       | desktop                                 | laptop                               |
|                     | ![Server](gpu_server.jpg)       | ![Desktop](alienware-area51.png) | ![Laptop](macpro_inside.png)  |
| Main usage          | scientific computing                   | daily work, gaming                      | daily work                           |
| Memory              | 80 GB                                    | 48 GB                                   | 16 GB                                  |
| Memory bandwidth    | 2 TB/sec                              | 960 GB/sec                               | 576 GB/sec                             |
| Number of cores     | ???                                    | ???                                     | ???                                  |
| Processor clock     | ??? GHz                                 | ??? GHz                                  | ??? GHz                               |
| Peak DP performance | 26 TFLOPS                              | ??? TFLOPS                                        |                                    ??? TFLOPS  |
| Peak SP performance | 51 TFLOPS                            | 91.1 TFLOPS                              | 42.6 TFLOPS                            |

## GPU architecture vs CPU architecture

* GPUs contain 1000s of processing cores on a single card; several cards can fit in a desktop PC  

* Each core carries out the same operations in parallel on different input data -- single program, multiple data (SPMD) paradigm  

* Extremely high arithmetic intensity *if* one can transfer the data onto and results off of the processors quickly

| ![i7 die](cpu_i7_die.png) | ![Fermi die](Fermi_Die.png) |
|----------------------------------|------------------------------------|
| ![Einstein](einstein.png) | ![Rain man](rainman.png)    |

## GPGPU in Julia

GPU support by Julia is under active development. Check [JuliaGPU](https://github.com/JuliaGPU) for currently available packages. 

There are multiple paradigms to program GPU in Julia, depending on the specific hardware.

- **CUDA** is an ecosystem exclusively for Nvidia GPUs. There are extensive CUDA libraries for scientific computing: CuBLAS, CuRAND, CuSparse, CuSolve, CuDNN, ...

  The [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl) package allows defining arrays on **Nvidia GPUs** and overloads many common operations.

- The [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl) package allows defining arrays on **AMD GPUs** and overloads many common operations.

- The [Metal.jl](https://github.com/JuliaGPU/Metal.jl) package allows defining arrays on **Apple Silicon** and overloads many common operations.

- The [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) package allows defining arrays on **Intel GPUs** and overloads many common operations.

I'll illustrate using Metal.jl on my MacBook Pro running MacOS Sequoia 15.4. It has Apple M2 chip with 38 GPU cores.

In [1]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/slides/09-juliagpu`
    Updating registry at `~/.julia/registries/General.toml`
   Installed GPUCompiler ── v1.3.2
   Installed OffsetArrays ─ v1.17.0
   Installed Atomix ─────── v1.1.1
    Updating `~/Documents/github.com/ucla-biostat-257/2025spring/slides/09-juliagpu/Project.toml`
  [13e28ba4] + AppleAccelerate v0.4.0
  [6e4b80f9] + BenchmarkTools v1.6.0
  [bdcacae8] + LoopVectorization v0.12.172
  [dde4c033] + Metal v1.5.1
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Documents/github.com/ucla-biostat-257/2025spring/slides/09-juliagpu/Manifest.toml`
  [79e6a3ab] + Adapt v4.3.0
  [13e28ba4] + AppleAccelerate v0.4.0
  [4fba245c] + ArrayInterface v7.18.0
  [a9b6321e] + Atomix v1.1.1
  [6e4b80f9] + BenchmarkTools v1.6.0
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.6
  [fa961155] + CEnum v0.5.0
  [2a0fbf3d] + CPUSummary v0.2.6
  [fb6a15b2] + CloseOpenIntervals v0.1.13
  [523fee87] + CodecBzip2 v0

Status `~/Documents/github.com/ucla-biostat-257/2025spring/slides/09-juliagpu/Project.toml`
  [13e28ba4] AppleAccelerate v0.4.0
  [6e4b80f9] BenchmarkTools v1.6.0
  [bdcacae8] LoopVectorization v0.12.172
  [dde4c033] Metal v1.5.1
  [37e2e46d] LinearAlgebra v1.11.0


   1432.4 ms  ✓ Atomix → AtomixMetalExt
  25 dependencies successfully precompiled in 23 seconds. 71 already precompiled.


## Query GPU devices in the system

In [4]:
using Metal, AppleAccelerate

Metal.versioninfo()

macOS 15.4.0, Darwin 24.4.0

Toolchain:
- Julia: 1.11.5
- LLVM: 16.0.6

Julia packages: 
- Metal.jl: 1.5.1
- GPUArrays: 11.2.2
- GPUCompiler: 1.3.2
- KernelAbstractions: 0.9.34
- ObjectiveC: 3.4.1
- LLVM: 9.2.0
- LLVMDowngrader_jll: 0.6.0+0

1 device:
- Apple M2 Max (64.000 KiB allocated)


## Transfer data between main memory and GPU

In [5]:
using Random
Random.seed!(257)

# generate SP data on CPU
x = rand(Float32, 3, 3)
# transfer data form CPU to GPU
xd = MtlArray(x)

3×3 MtlMatrix{Float32, Metal.PrivateStorage}:
 0.145793  0.939801  0.479926
 0.567772  0.577251  0.81655
 0.800538  0.38893   0.914135

In [6]:
# generate array on GPU directly
# yd = Metal.ones(3, 3)
yd = MtlArray(ones(Float32, 3, 3))

3×3 MtlMatrix{Float32, Metal.PrivateStorage}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [7]:
# collect data from GPU to CPU
x = collect(xd)

3×3 Matrix{Float32}:
 0.145793  0.939801  0.479926
 0.567772  0.577251  0.81655
 0.800538  0.38893   0.914135

## Linear algebra

In [8]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(257)

n = 2^14
# on CPU
x = rand(Float32, n, n)
y = rand(Float32, n, n)
z = zeros(Float32, n, n)
# on GPU
xd = MtlArray(x)
yd = MtlArray(y)
zd = MtlArray(z);

### Dot product

In [9]:
# SP matrix dot product on GPU: tr(X'Y)
# why are there allocations?
bm_gpu = @benchmark Metal.@sync dot($xd, $yd)

BenchmarkTools.Trial: 656 samples with 1 evaluation per sample.
 Range (min … max):  7.451 ms …   9.082 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.577 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.604 ms ± 131.507 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▁▄▄█▅▅▇▂ ▂▃▆▃▂▃ ▁                                         
  ▃▅████████████████▇█▇▅▆▅▅▄▄▃▄▄▃▄▄▂▄▃▃▄▃▃▃▂▂▁▃▃▃▃▃▁▂▃▃▂▁▁▁▂▂ ▄
  7.45 ms         Histogram: frequency by time        8.01 ms <

 Memory estimate: 21.24 KiB, allocs estimate: 837.

In [10]:
# SP matrix dot product on CPU: tr(X'Y)
bm_cpu = @benchmark dot($x, $y)

BenchmarkTools.Trial: 144 samples with 1 evaluation per sample.
 Range (min … max):  34.328 ms …  38.189 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     34.654 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   34.811 ms ± 554.004 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂  ▂▄█▃▂▆                                                   
  ▅▃█▆███████▇▆▆▄▃▁▃▄▁▁▁▁▁▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▃▄▁▄▁▃▃▁▁▁▃▃▁▁▁▃ ▃
  34.3 ms         Histogram: frequency by time         36.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
# speedup
median(bm_cpu.times) / median(bm_gpu.times)

4.57366119419473

### Broadcast

In [12]:
# SP broadcast on GPU: z .= x .* y
# why is there allocation?
bm_gpu = @benchmark Metal.@sync $zd .= $xd .* $yd

BenchmarkTools.Trial: 542 samples with 1 evaluation per sample.
 Range (min … max):  8.788 ms …  11.013 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.094 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.216 ms ± 367.317 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▅█▆▇▅▆ ▂▁       ▁                                        
  ▅▄▆███████▇██▇▇▆▆██▅█▇▇█▆▇▆▂▃▂▃▂▂▂▂▁▂▁▁▁▁▁▁▁▂▂▃▃▄▃▃▅▃▂▃▂▃▃▃ ▄
  8.79 ms         Histogram: frequency by time        10.4 ms <

 Memory estimate: 4.53 KiB, allocs estimate: 177.

In [13]:
# SP broadcast on CPU: z .= x .* y
bm_cpu = @benchmark $z .= $x .* $y

BenchmarkTools.Trial: 138 samples with 1 evaluation per sample.
 Range (min … max):  35.229 ms … 39.946 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     35.947 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.313 ms ±  1.094 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂▃▁▇█▆▆ ▄▃ ▄                                               
  ▆▄████████████▇▃▄▃▁▄▃▁▃▁▆▁▁▄▁▃▁▃▁▁▃▁▁▃▁▁▁▁▁▁▃▃▃▃▃▁▁▁▁▃▁▃▃▄▃ ▃
  35.2 ms         Histogram: frequency by time        39.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
# speedup
median(bm_cpu.times) / median(bm_gpu.times)

3.9528966986700036

### Matrix multiplication

In [15]:
# SP matrix multiplication on GPU
bm_gpu = @benchmark Metal.@sync mul!($zd, $xd, $yd)

BenchmarkTools.Trial: 6 samples with 1 evaluation per sample.
 Range (min … max):  908.557 ms … 956.545 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     913.286 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   921.531 ms ±  18.726 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁        ▁              ▁                                  ▁  
  ██▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  909 ms           Histogram: frequency by time          957 ms <

 Memory estimate: 960 bytes, allocs estimate: 54.

For this problem size on this machine, we see GPU achieves a staggering **9 TFLOPS** throughput with single precision!

In [17]:
# SP throughput on GPU
(2n^3) / (minimum(bm_gpu.times) / 1e9)

9.681393531616361e12

In [18]:
# SP matrix multiplication on CPU
bm_cpu = @benchmark mul!($z, $x, $y)

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  4.038 s …   4.065 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.052 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.052 s ± 19.188 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.04 s         Histogram: frequency by time        4.07 s <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [19]:
# SP throughput on CPU
(2n^3) / (minimum(bm_cpu.times) / 1e9)

2.1781870131836782e12

We see >10x speedup by GPUs in this matrix multiplication example.

In [ ]:
# cholesky on Gram matrix
# This one doesn't seem to work on Apple M2 chip yet
xtxd = xd'xd + I
@benchmark Metal.@sync cholesky($(xtxd))

ArgumentError: ArgumentError: cannot take the CPU address of a MtlMatrix{Float32, Metal.PrivateStorage}

In [32]:
xtx = collect(xtxd)
@benchmark LinearAlgebra.cholesky($(Symmetric(xtx)))

BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 7.655 s (0.01% GC) to evaluate,
 with a memory estimate of 1.00 GiB, over 3 allocations.

In [ ]:
@benchmark AppleAccelerate.cholesky($(Symmetric(xtx)))

We don't see GPU speedup of Cholesky at the moment.

## Evaluation of elementary and special functions on GPU

In [ ]:
# elementwise function on GPU arrays
fill!(yd, 1)
bm_gpu = @benchmark Metal.@sync $zd .= log.($yd .+ sin.($xd))
bm_gpu

In [ ]:
# elementwise function on CPU arrays
x, y, z = collect(xd), collect(yd), collect(zd)
bm_cpu = @benchmark $z .= log.($y .+ sin.($x))
bm_cpu

In [ ]:
# Speed up
median(bm_cpu.times) / median(bm_gpu.times)

GPU brings great speedup (>50x) to the massive evaluation of elementary math functions.